# Report:
## Where to place a new accomodation business in the historic center of Rome?

Andrea Bucci

### Introduction/Business Problem

In this project, I explore the neighborhoods of the 1st "Municipio" (equivalent to Borough) of Rome, Italy. This is the historic heart of the city and contains 22 Rioni (neighborhoods). An enterpreneur wants to know which is the best location where to start a business regarding tourist accomodation; he needs a map showing the existing businesses around the Rioni and the tourist attractions distribution so to get a visual idea about a potentially "unworked" area. Furthermore, he wants to have a look at each Rione's top 10 most occurrent venues and to weight his final judgment with the demographics numbers so that, along with existing target businesses, he could have an idea of the number of private citizens living in the area and eventually renting out flats or rooms to tourists.

### Data

The data we are looking for are distributed over mainly three sources. 
The <b>first</b> being a kml file taken from a custom map on google mymaps, containing the geographical coordinates for each Rione; we decided not to use any specific kml/xml parsing library (as pykml,lxml,etc.) but parse the file with some self-made code. There will be three final columns: the name of the Rione, its latitude and its longitude.
The <b>second</b> source is a Wikipedia page containing info about the number of residents, area (expressed in squared kilometers) and population density (expressed in number of people per squared kilometer) in each Rione. We will mantain all of these information for some final evaluations about the place chosen to install the accomodation business.
The <b>third</b> and last piece of information will be gotten from Foursquare, by getting all the venues nearby our neighborhoods location in a 500 meters radius and with a limit of 100 venues. These info will comprise the individual venue's name, latitude and longitude, category.

A first dataframe will look like this:

<img src='df1.png' alt="df1" width="300"/>

The second one, joining the first with the demographics will be :

<img src='df2.png' alt="df2" width="450" />

And finally a concatenation of the first with the Foursquare requests to the first will be created:

<img src='romevenues.png' alt="romevenues" width="700" />

### Methodology

First of all, we use Geopy to get the coordinates for Rome, Italy. Then, with Folium, we can easily set up a map centered in Rome and having a close-up zoom on its historic center, where all the Rioni are located.
<img src='riomap.png' alt="riomap" width="600"/>
Now we can gather venues data from Foursquare; after the credentials set up we send a get request for the first 100 venues in a 500 meter radius around the first Rione's coordinates: Monti. The JSON file we get in response will be used to harvest all the information we need about venues. The curated dataframe that we get appears like this:
<img src='montidf.png' alt="montidf" width="400"/>
We now can code a small cycle which will do the job already done with Rione Monti for all the Rioni in the 1st Municipality. This brings us to have a dataframe, called <b>rome_venues</b>,with all the venues name, location and category,other than their belonging Rione.
<img src='romevenues.png' alt="romevenues" width="500"/>
From this last dataframe we can get and visualize target businesses (Hotels,B&Bs,Hostels) and touristic points of interests on two different maps, using subsets of the known dataframe. The maps would look like these:
<img src='tarbusmap.png' alt="tarbusmap" width="400"/> <img src='poimap.png' alt="poimap" width="400"/>
This gives a first glance at the distribution of both the existing businesses that could become competitors to the entrepreneur and all the venues that would be needed to be as closer as possible to the new business location.
From the first subdataframe, the one pertaining the hotels, we can leave just the coordinates of the venues location so to be possible for us to cluster them geographically. To accomplish this, we use <b>K-Means</b> algorithm included in the ScikitLearn library. K-Means aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean (cluster centers or cluster centroid). Before fitting the data, an analysis regarding the optimal choice of k is just preferred in this scenario, but fundamental in others. 
The <b>Elbow method</b> already gives us a neat response but to analyze this further, we decide to use the Silhouette Score method as well. In cluster analysis, the elbow method is a heuristic used in determining the number of clusters in a data set. The method consists of plotting the explained variation as a function of the number of clusters, and picking the elbow of the curve as the number of clusters to use. The silhouette value is a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation). The silhouette ranges from −1 to +1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters. The metric used in our case to calculate distances is Euclidean.
<img src='elbow.png' alt="elbow" width="400"/> <img src='silhou.png' alt="silhou" width="400"/>
Since the clustering would only give us a visual grouped distribution of the target businesses, we do not stricly rely on the anyway useful methods above, and decide to set the number of clusters to 10. This is the resulting map of the clusters:
<img src='clustermap.png' alt="clustermap" width="500"/>
To have a clearer view of the edges between clusters, we choose to draw a Voronoi diagram and superimpose it to the map. A Voronoi diagram is a partition of a plane into regions close to each of a given set of objects (in our case, the clusters centroids). These regions are called Voronoi cells.
<img src='vorunb.png' alt="vorunb" width="400"/>
So this is what our clusters look like when closed inside sectors, the Voronoi cells. We can see the centroids represented as blue dots, sectors vertices as yellow dots and sectors edges as continous lines, but we also notice the presence of dashed lines, indicating that some sectors are not closed (the dashed line is an infinite line ad not a side of the polygonal sector).
Therefore, we may want to set a <b>bounding_box</b> that artificially gives an end to those lines. To plot this modified version of the Voronoi Diagram, we have to build a function that could integrate the above bounds to the original figure.
<img src='vorb.png' alt="vorb" width="400"/>
Now that we have the sectors all bounded, we can get back the map of clusters we saw earlier and superimpose the diagram. Also, we will add the p.o.i.s on the map to have a complete picture.
<img src='finmap.png' alt="finmap" width="500"/>
Simply by visually inspecting this map (even though lots of further computed analyses could be done) the entrepreneur decides to set his potential new business in a specific region between Rione Campitelli and Rione Sant'Angelo. This optimal zone is then added to the map.
<img src='optareamap.png' alt="optareamap" width="400"/>

### Results


The entrepreneur is pretty satisfied by the analyses done until now. He clearly saw how that area between the two above-cited neighborhoods of Central Rome could be the target location of his investment because of the vicinity to the majority of touristic p.o.i. and the relative scarcity of hotels or similar businesses. Moreover, its position is really more central with respect to other sparse areas around our bounding box. One final factor influencing the decision is the proximity to the Train Central Station of Termini (you can see it in between two green clusters on the right. Close enough to it to allow fast shuttling between it and his business but not so close to it to be overwhelmed by all the concurrency nearby. Also, in that case the tourist attractions would have been really further away and not pleasably reached on foot.
Nevertheless, the entrepreneur would like to have a glance at some final information about number of specific venues inside the Rioni and around his optimal zone, other than retrieving some general demographics and gather if his chosen area would be more "immune" than others to privates renting out their own places or parts of them.
First of all, a rapid look at the 5 and 10 most common venues in each Rione is requested, and even if we could have just counted the venues from the <b>rome_venues</b> dataframe built before, we would like to convert that into a "one-hot-encoded" new dataframe
that could serve for some afterthoughts of the entrepreneur being translated into analysis.
Interestingly enough, the 5 most common venues throughout all of the Rioni but the two chosen for the optimal zone (Campitelli and Sant'Angelo) have a massive presence of Italian Restaurants, Cafès, Ice-Cream Shops and Hotels. These latters instead show a great countertrend having mostly historic sites at the top occurence frequency and very low number of accomodations.
We extend the investigation to have a look at the top 10 most common venues.
<img src='tp10.png' alt="top10" width="700"/>

### Discussion

This extended view reinforces the good feeling of the entrepreneur regarding his choice in terms of scarcity of existing accomodations in the two target neighborhoods. The relative small presence of restaurants other than italian ones goes in the same direction even if a more robust presence of traditional roman Osteria would have been appreciated.
From the demographic point of view puts instead our optimal zone in a low-medium class for number of residents and density over a squared kilometer, thus being very good to lower the private competition.
Some final counts of total venues, points of interest, museums and roman restaurants per Rione are calculated as final evaluation.
<img src='counts.png' alt='counts' width='400'/> 
<img src='countr.png' alt='countr' width='400'/> 
As expected, the roman traditional cuisine is not so much present in the target neighborhoods, but being very close to neighborhoods that have a big number of them, this is not considered a problem at all. On the other hand, the remaining counts give a medium to high leverage to the target neighborhoods.

### Conclusion

These last verifications were only meant to complete the information regarding the optimal zone with respect to the rest of First Municipality of Rome. The choice remains almost the same we explored right after the clustering above, with a slight inclination to build along the Voronoi ridge that we see crossing the right side of the optimal zone almost south-north direction, because of the proximity to the touristic attractions and roman restaurants in nearby neighborhoods.